# FPL Bot V1

## Plan

### Start Simple

The big ambition of an FPL bot is to produce a predictive model capable of looking ahead at future fixtures, predicting how many points each player is going to score, and optimising the team selection over the coming weeks. However, there are so many moving parts in such a bot, that deploying something capable of doing all of this will take a long time. Ther best way to start is to start, and to start simple at that.

Therefore, the plan for the first iteration of this model is to simply look at the number of points scored over the past 5 gameweeks, and optimising (in whatever fashion this turns out to be) for player price. This model is unlikely to be particularly groundbreraking, but there is still a lot of ground to be covered in terms of researching appropriate datasets, scraping data, analysing players, team selection in accordance with constraints, and optimising player selection.

## Data Sources

A quick google search found a useful reddit sub (https://www.reddit.com/r/FantasyPL/comments/10rxb2j/websites_for_fpl_data_analysis/), which listed a number of useful websites for playing FPL. Of these, the more useful looking ones are listed below:
- https://www.fpl-data.co.uk/statistics
- https://fplform.com/fpl-player-data
- https://www.fplanalytics.com/playerStatus.html
- https://fantasy.premierleague.com/statistics
- https://understat.com/league/EPL

Understat is one I've come across before, which summarieses (really nicely) expected goal data. Perhaps the only issue with it is that it isn't FPL related, so doesn't include FPL points data. Conveniently, FPL-Data sources some of its data from understat, but importantly also contains points data too. So this is the first choice of data source for this simple FPL bot.

## Web Scraping

I am following the web-scraping 'tutorial' in the following website: https://www.learndatasci.com/tutorials/ultimate-guide-web-scraping-w-python-requests-and-beautifulsoup/. 

First things first, we need to make a request to the URL:

In [9]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.fpl-data.co.uk/statistics'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [10]:
soup


<!DOCTYPE html>

<html>
<head>
<script>
            var host = "www.fpl-data.co.uk" || "fpl-data.co.uk";
            if ((host == window.location.host) && (window.location.protocol != "https:")){
            window.location.protocol = "https";
            }
        </script>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-3VHMZFN5JM"></script>
<script>
          window.dataLayer = window.dataLayer || [];
          function gtag(){dataLayer.push(arguments);}
          gtag('js', new Date());
        
          gtag('config', 'G-3VHMZFN5JM');
        </script>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>FPL Data</title>
<link href="/assets/favicon.ico?m=1727784854.0" rel="icon" type="image/x-icon"/>
<link href="/assets/style.css?m=1727784854.0" rel="stylesheet"/>
</head>
<body>
<div id="react-entry-point

In [11]:
rows = soup.select('tabindex tr')
rows

[]

### Dynamic Loaded Data

Clearly from above there is an issue trying to parse the data using beautiful soup. The main bulk of the data I want to be able to access is not showing, instead all that is being displayed in the `soup` object is "Loading...". This suggests that the data is being loaded dynamically.

The following stack overflow page has some good info on dynamically loaded data https://stackoverflow.com/questions/55709463/how-to-scrape-dynamic-content-from-a-website. "Dynamic means the data is generated from a request after the initial page request. Static means all the data is there at the original call to the site"

According to the above page, Selenium is a good option for loading dynamic data. It allows you to "simulate opening a browser, letting the page render, then pull the html source code".

### Selenium

The following page provides a tutorial to web scrape using selenium, which is what I am following for this example: https://serpapi.com/blog/selenium-web-scraping-python/

In [12]:
# Step 1: Install Selenium
# pip install selenium

In [13]:
# Step 2: Download a WebDriver
# A WebDriver is needed to automate a browser
# For Chrome, ChromeDriver is used (https://storage.googleapis.com/chrome-for-testing-public/129.0.6668.70/win64/chromedriver-win64.zip)
# The ChromeDriver must match the Chrome version (Version 129.0.6668.71 (Official Build) (64-bit))
# The ChromeDriver location must be known (C:\Program Files\chromedriver-win64)

In [14]:
# Step 3: Import Selenium and Initialise WebDriver
from selenium import webdriver

driver = webdriver.Chrome()

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


In [15]:
# Step 4: Sample running browser
# Use the following URL as an example: https://www.scrapethissite.com/pages/forms

url = 'https://www.scrapethissite.com/pages/forms'
driver.get(url)

# Print title
print(driver.title)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x00007FF6DFCEB125+29573]
	(No symbol) [0x00007FF6DFC5FF50]
	(No symbol) [0x00007FF6DFB1B6EA]
	(No symbol) [0x00007FF6DFAEFCD5]
	(No symbol) [0x00007FF6DFB9EF67]
	(No symbol) [0x00007FF6DFBB7FC1]
	(No symbol) [0x00007FF6DFB970A3]
	(No symbol) [0x00007FF6DFB612DF]
	(No symbol) [0x00007FF6DFB62441]
	GetHandleVerifier [0x00007FF6E001C76D+3377613]
	GetHandleVerifier [0x00007FF6E0067B67+3685831]
	GetHandleVerifier [0x00007FF6E005CF8B+3641835]
	GetHandleVerifier [0x00007FF6DFDAB2A6+816390]
	(No symbol) [0x00007FF6DFC6B25F]
	(No symbol) [0x00007FF6DFC67084]
	(No symbol) [0x00007FF6DFC67220]
	(No symbol) [0x00007FF6DFC5607F]
	BaseThreadInitThunk [0x00007FFDF6B753E0+16]
	RtlUserThreadStart [0x00007FFDF7B6485B+43]


In [8]:
# Step 5: Interact with the page
# On the page there is a search box with the id "q"
# We will search for "kings"

# find q
q = driver.find_element("id", "q")
# fill with keyword "kings"
q.send_keys("kings")
# submit
q.submit()

# read current url
print(driver.current_url)

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x00007FF6DFCEB125+29573]
	(No symbol) [0x00007FF6DFC5FF50]
	(No symbol) [0x00007FF6DFB1B6EA]
	(No symbol) [0x00007FF6DFB041E7]
	(No symbol) [0x00007FF6DFB03EEA]
	(No symbol) [0x00007FF6DFB01D65]
	(No symbol) [0x00007FF6DFB025BF]
	(No symbol) [0x00007FF6DFB110CE]
	(No symbol) [0x00007FF6DFB29B8D]
	(No symbol) [0x00007FF6DFB2F71A]
	(No symbol) [0x00007FF6DFB02D24]
	(No symbol) [0x00007FF6DFB29972]
	(No symbol) [0x00007FF6DFBB8A01]
	(No symbol) [0x00007FF6DFB970A3]
	(No symbol) [0x00007FF6DFB612DF]
	(No symbol) [0x00007FF6DFB62441]
	GetHandleVerifier [0x00007FF6E001C76D+3377613]
	GetHandleVerifier [0x00007FF6E0067B67+3685831]
	GetHandleVerifier [0x00007FF6E005CF8B+3641835]
	GetHandleVerifier [0x00007FF6DFDAB2A6+816390]
	(No symbol) [0x00007FF6DFC6B25F]
	(No symbol) [0x00007FF6DFC67084]
	(No symbol) [0x00007FF6DFC67220]
	(No symbol) 

In [44]:
# Step 6: Print content
# We want to print the table content 

from selenium.webdriver.common.by import By

# Print all table values
table = driver.find_element(By.CLASS_NAME, "table")
print(table.text)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".table"}
  (Session info: chrome=129.0.6668.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DFCEB125+29573]
	(No symbol) [0x00007FF6DFC5FF50]
	(No symbol) [0x00007FF6DFB1B6EA]
	(No symbol) [0x00007FF6DFB6F815]
	(No symbol) [0x00007FF6DFB6FA6C]
	(No symbol) [0x00007FF6DFBBB917]
	(No symbol) [0x00007FF6DFB9733F]
	(No symbol) [0x00007FF6DFBB86BC]
	(No symbol) [0x00007FF6DFB970A3]
	(No symbol) [0x00007FF6DFB612DF]
	(No symbol) [0x00007FF6DFB62441]
	GetHandleVerifier [0x00007FF6E001C76D+3377613]
	GetHandleVerifier [0x00007FF6E0067B67+3685831]
	GetHandleVerifier [0x00007FF6E005CF8B+3641835]
	GetHandleVerifier [0x00007FF6DFDAB2A6+816390]
	(No symbol) [0x00007FF6DFC6B25F]
	(No symbol) [0x00007FF6DFC67084]
	(No symbol) [0x00007FF6DFC67220]
	(

In [17]:
# Step 7: Close the browser
# Once we are done we must close the browser
# If we close the browser before we are done we will get an error

driver.quit()

## Back to FPL Data

Having learnt how to use Selenium for dynamic content, I will use it to load data from https://www.fpl-data.co.uk/statistics

Clearly there is an isssue here, whereby only the first column is being loaded.

The following webpage provides a tutorial for loading data from a dynamic table using Selenium, which is being used here.

https://www.lambdatest.com/blog/how-to-handle-web-table-in-selenium-webdriver/

In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd

def load_data():
    """
    Loads the data from the player statistics table and stores in a pandas dataframe. The table is spread over 35 pages so this 
    function iterates over each page. 

    Approx runtime: 7 mins
    """
    # Load the web page
    driver = webdriver.Chrome()

    url = 'https://www.fpl-data.co.uk/statistics'
    driver.get(url)

    # Wait until the web page is fully loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="stats-data-table"]/div[3]/div/div[2]'))).click()

    # Find the highest page number in the table
    max_page = int(driver.find_element('xpath', '/html/body/div/div/div[4]/div/div[8]/div[3]/div/div[2]').text)
    
    # Select the column names from the table
    head_row = driver.find_element("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[2]').text
    cols_for_df = ['Player']
    cols_for_df.extend(head_row.split('\n'))

    # Initialise the master dataframe
    df_master = pd.DataFrame(columns=cols_for_df)

    # Define X paths
    before_XPath = '//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr['
    aftertd_XPath = ']/td['
    aftertr_XPath = ']'

    # Define the number of rows/columns in the table
    n_rows = len(driver.find_elements("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr'))
    n_columns = len(driver.find_elements("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[3]/td'))
    
    # Iterate over every page in the table
    for page_num in range(max_page):
        # Change page number
        current_page = driver.find_element(By.CLASS_NAME, 'current-page')
        current_page.send_keys(f"{page_num+1}")
        current_page.send_keys(Keys.RETURN)

        # Initialise rows (which holds all rows per page) and break indicator (whichindicates when end of table has been reached) parameters
        rows = []
        break_ind = False

        # Iterate over every row
        for t_row in range(3, (n_rows + 1)):
            # Initialise row_text parameter, which holds one full row of data
            row_text = []
            # Iterate over every column
            for t_column in range(1, (n_columns + 1)):
                # Define xpath of given cell
                FinalXPath = before_XPath + str(t_row) + aftertd_XPath + str(t_column) + aftertr_XPath
                
                # Try to access cell, handling "NoSuchElementException" in case end of table reached
                try:
                    cell_text = driver.find_element("xpath",FinalXPath).text
                except NoSuchElementException:
                    # If final page reached then break
                    if page_num == max_page - 1:
                        break_ind = True
                        break
                    else:
                        raise NoSuchElementException
                # Append cell to row_text parameter
                row_text.append(cell_text)
                
            # Append row_text to rows if end of table not reached (in which case row_text is full of nulls)
            if not break_ind:
                rows.append(row_text)
                    
        # Create dataframe containing all data in table page
        df_temp = pd.DataFrame(rows,columns=cols_for_df) 

        # For some reason, the above process does not access first column containing players, and the below process only accesses first column
        # Get the players from the first column and add them to the dataframe
        table = driver.find_element(By.CLASS_NAME, "cell-table")
        players = table.text.split('\n')
        players.remove(players[0])
        df_temp['Player'] = players

        # Concatenate the page specific dataframe to the master dataframe
        df_master = pd.concat([df_master,df_temp])

    # Close the browser when done
    driver.quit()
    return df_master.reset_index(drop=True)

# df = load_data()
# df.head(10)

## Filtering to last X games

The overall goal of this model is to pick the players who have scored the most over the past 5 gameweeks. Since at the time of writing (26/09/24) there have only been 5 gameweeks, I will test the following functionality to filter down to the past 3 gameweeks, with the ability to tweak this number via an input parameter.

In [19]:
from selenium.webdriver.support.ui import WebDriverWait 

driver = webdriver.Chrome()

url = 'https://www.fpl-data.co.uk/statistics'
driver.get(url)

WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="input-gameweek-stats"]/div/div[4]'))).click()

In [20]:
slider = driver.find_element(By.XPATH, '//*[@id="input-gameweek-stats"]/div/div[4]')
val_max = int(slider.get_attribute('aria-valuemax'))
val_now = int(slider.get_attribute('aria-valuenow'))

while val_max - val_now > 2:
    print(val_max,val_now)
    slider.send_keys(Keys.ARROW_RIGHT)
    slider.send_keys(Keys.RETURN)
    val_now = int(slider.get_attribute('aria-valuenow'))

driver.quit()

6 1
6 2
6 3


In [21]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd

def load_data(num_gws=3):
    """
    Loads the data from the player statistics table and stores in a pandas dataframe. The table is spread over 35 pages so this 
    function iterates over each page. 

    Approx runtime: 7 mins
    """
    # Load the web page
    driver = webdriver.Chrome()

    url = 'https://www.fpl-data.co.uk/statistics'
    driver.get(url)

    # Wait until the web page is fully loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="input-gameweek-stats"]/div/div[4]'))).click()

    slider = driver.find_element(By.XPATH, '//*[@id="input-gameweek-stats"]/div/div[4]')
    val_max = int(slider.get_attribute('aria-valuemax'))
    val_now = int(slider.get_attribute('aria-valuenow'))

    while val_max - val_now > (num_gws-1):
        slider.send_keys(Keys.ARROW_RIGHT)
        slider.send_keys(Keys.RETURN)
        val_now = int(slider.get_attribute('aria-valuenow'))

    # Find the highest page number in the table
    max_page = int(driver.find_element('xpath', '/html/body/div/div/div[4]/div/div[8]/div[3]/div/div[2]').text)
    
    # Select the column names from the table
    head_row = driver.find_element("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[2]').text
    cols_for_df = ['Player']
    cols_for_df.extend(head_row.split('\n'))

    # Initialise the master dataframe
    df_master = pd.DataFrame(columns=cols_for_df)

    # Define X paths
    before_XPath = '//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr['
    aftertd_XPath = ']/td['
    aftertr_XPath = ']'

    # Define the number of rows/columns in the table
    n_rows = len(driver.find_elements("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr'))
    n_columns = len(driver.find_elements("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[3]/td'))
    
    # Iterate over every page in the table
    for page_num in range(max_page):
        # Change page number
        current_page = driver.find_element(By.CLASS_NAME, 'current-page')
        current_page.send_keys(f"{page_num+1}")
        current_page.send_keys(Keys.RETURN)

        # Initialise rows (which holds all rows per page) and break indicator (whichindicates when end of table has been reached) parameters
        rows = []
        break_ind = False

        # Iterate over every row
        for t_row in range(3, (n_rows + 1)):
            # Initialise row_text parameter, which holds one full row of data
            row_text = []
            # Iterate over every column
            for t_column in range(1, (n_columns + 1)):
                # Define xpath of given cell
                FinalXPath = before_XPath + str(t_row) + aftertd_XPath + str(t_column) + aftertr_XPath
                
                # Try to access cell, handling "NoSuchElementException" in case end of table reached
                try:
                    cell_text = driver.find_element("xpath",FinalXPath).text
                except NoSuchElementException:
                    # If final page reached then break
                    if page_num == max_page - 1:
                        break_ind = True
                        break
                    else:
                        raise NoSuchElementException
                # Append cell to row_text parameter
                row_text.append(cell_text)
                
            # Append row_text to rows if end of table not reached (in which case row_text is full of nulls)
            if not break_ind:
                rows.append(row_text)
                    
        # Create dataframe containing all data in table page
        df_temp = pd.DataFrame(rows,columns=cols_for_df) 

        # For some reason, the above process does not access first column containing players, and the below process only accesses first column
        # Get the players from the first column and add them to the dataframe
        table = driver.find_element(By.CLASS_NAME, "cell-table")
        players = table.text.split('\n')
        players.remove(players[0])
        df_temp['Player'] = players

        # Concatenate the page specific dataframe to the master dataframe
        df_master = pd.concat([df_master,df_temp])

    # Close the browser when done
    driver.quit()
    return df_master.reset_index(drop=True)

# df = load_data(num_gws=3)
# df.head(10)

## Team Selection

Now that the raw dataset has been sourced, a team selection function is required that can scour the dataset and choose the best team based on a provided metric. In this case we are looking at points scored over the past 3 weeks. The team selection function must select the 15 player squad that fits to the constraints (2 GKs, 5 DEFs, 5 MIDs, 3 ATTs) while maximising the overall number of players.

In this simple example it suffices to order the dataset by total points, add each player unless the max number of players in their position has been filled, and continue until the team has been filled.

The issue I have here is that the dataset in its current form does not display a players position. I considered scraping this data from another source, but this proved too difficult. A simpler option is to run the `load_data` function above 4 times filtered for each position, resulting in a gks dataset, a defs dataset, a mids dataset, and an atts dataset. I can then create a new `position` column and set the value depending on the dataset. Finally these can simply be concatenated into one dataset.

Note, in this first iteration the price of a player is not being considered. This will be added later.

In [22]:
driver = webdriver.Chrome()

url = 'https://www.fpl-data.co.uk/statistics'
driver.get(url)

In [23]:
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="input-position-team-stats"]/label[1]/input')))
gk_checkbox = driver.find_element(By.XPATH, '//*[@id="input-position-team-stats"]/label[1]/input')
not gk_checkbox.is_selected()

False

In [24]:
driver.quit()

In [25]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pandas as pd

def load_data_by_position(num_gws,position):
    """
    Loads the data from the player statistics table and stores in a pandas dataframe. The table is spread over 35 pages so this 
    function iterates over each page. 

    Approx runtime: 8.5 mins
    """
    ### Load the web page
    driver = webdriver.Chrome()

    url = 'https://www.fpl-data.co.uk/statistics'
    driver.get(url)

    # Wait until the web page is fully loaded
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="input-gameweek-stats"]/div/div[4]'))).click()

    slider = driver.find_element(By.XPATH, '//*[@id="input-gameweek-stats"]/div/div[4]')
    val_max = int(slider.get_attribute('aria-valuemax'))
    val_now = int(slider.get_attribute('aria-valuenow'))

    while val_max - val_now > (num_gws-1):
        slider.send_keys(Keys.ARROW_RIGHT)
        slider.send_keys(Keys.RETURN)
        val_now = int(slider.get_attribute('aria-valuenow'))

    ### Filter the webpage by position
    # Define position checkboxes
    gk_checkbox = driver.find_element(By.XPATH, '//*[@id="input-position-team-stats"]/label[1]/input')
    def_checkbox = driver.find_element(By.XPATH, '//*[@id="input-position-team-stats"]/label[2]/input')
    mid_checkbox = driver.find_element(By.XPATH, '//*[@id="input-position-team-stats"]/label[3]/input')
    att_checkbox = driver.find_element(By.XPATH, '//*[@id="input-position-team-stats"]/label[4]/input')

    # Toggle checkboxes to only show desired data by position
    if position == 'gk':
        if not gk_checkbox.is_selected():
            gk_checkbox.click()
        for button in (def_checkbox, mid_checkbox, att_checkbox):
            if button.is_selected():
                button.click()
    elif position == 'def':
        if not def_checkbox.is_selected():
            def_checkbox.click()
        for button in (gk_checkbox, mid_checkbox, att_checkbox):
            if button.is_selected():
                button.click()    
    elif position == 'mid':
        if not mid_checkbox.is_selected():
            mid_checkbox.click()
        for button in (gk_checkbox, def_checkbox, att_checkbox):
            if button.is_selected():
                button.click()
    elif position == 'att':
        if not att_checkbox.is_selected():
            att_checkbox.click()
        for button in (gk_checkbox, def_checkbox, mid_checkbox):
            if button.is_selected():
                button.click()
    elif position == 'all':
        for button in (gk_checkbox, def_checkbox, mid_checkbox, att_checkbox):
            if not button.is_selected():
                button.click()
    else:
        raise NameError(f'Invalid position "{position}". Must be one of "gk", "def", "mid", "att", or "all".')

    # Find the highest page number in the table
    max_page = int(driver.find_element('xpath', '/html/body/div/div/div[4]/div/div[8]/div[3]/div/div[2]').text)
    
    # Select the column names from the table
    head_row = driver.find_element("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[2]').text
    cols_for_df = ['Player']
    cols_for_df.extend(head_row.split('\n'))

    # Initialise the master dataframe
    df_master = pd.DataFrame(columns=cols_for_df)

    # Define X paths
    before_XPath = '//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr['
    aftertd_XPath = ']/td['
    aftertr_XPath = ']'

    # Define the number of rows/columns in the table
    n_rows = len(driver.find_elements("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr'))
    n_columns = len(driver.find_elements("xpath",'//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[3]/td'))
    
    # Iterate over every page in the table
    for page_num in range(max_page):
        # Change page number
        current_page = driver.find_element(By.CLASS_NAME, 'current-page')
        current_page.send_keys(f"{page_num+1}")
        current_page.send_keys(Keys.RETURN)

        # Wait until new page loads fully
        # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="stats-data-table"]/div[2]/div/div[2]/div[2]/table/tbody/tr[3]/td')))

        # Initialise rows (which holds all rows per page) and break indicator (whichindicates when end of table has been reached) parameters
        rows = []
        break_ind = False

        # Iterate over every row
        for t_row in range(3, (n_rows + 1)):
            # Initialise row_text parameter, which holds one full row of data
            row_text = []
            # Iterate over every column
            for t_column in range(1, (n_columns + 1)):
                # Define xpath of given cell
                FinalXPath = before_XPath + str(t_row) + aftertd_XPath + str(t_column) + aftertr_XPath
                
                # Try to access cell, handling "NoSuchElementException" in case end of table reached
                try:
                    # cell_text = driver.find_element("xpath",FinalXPath).text
                    cell_text = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, FinalXPath))).text
                except TimeoutException:
                    break_ind = True
                    break
                # Append cell to row_text parameter
                row_text.append(cell_text)
                
            # Append row_text to rows if end of table not reached (in which case row_text is full of nulls)
            if break_ind:
                break
            else:
                rows.append(row_text)
                    
        # Create dataframe containing all data in table page
        df_temp = pd.DataFrame(rows,columns=cols_for_df) 

        # For some reason, the above process does not access first column containing players, and the below process only accesses first column
        # Get the players from the first column and add them to the dataframe
        table = driver.find_element(By.CLASS_NAME, "cell-table")
        players = table.text.split('\n')
        players.remove(players[0])
        df_temp['Player'] = players

        # Concatenate the page specific dataframe to the master dataframe
        df_master = pd.concat([df_master,df_temp])

    # Close the browser when done
    driver.quit()
    return df_master.reset_index(drop=True)

def load_data(num_gws):
    positions = ['gk','def','mid','att']

    df_master = pd.DataFrame()
    for position in positions:
        df_temp = load_data_by_position(num_gws,position)
        df_temp['Pos'] = position
        if df_master.empty:
            df_master = df_temp
        else:
            df_master = pd.concat([df_master,df_temp])

    return df_master

df = load_data(num_gws=5)
df.head(10)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6DFCEB125+29573]
	(No symbol) [0x00007FF6DFC5FF50]
	(No symbol) [0x00007FF6DFB1B6EA]
	(No symbol) [0x00007FF6DFB6F815]
	(No symbol) [0x00007FF6DFB6FA6C]
	(No symbol) [0x00007FF6DFBBB917]
	(No symbol) [0x00007FF6DFB9733F]
	(No symbol) [0x00007FF6DFBB86BC]
	(No symbol) [0x00007FF6DFB970A3]
	(No symbol) [0x00007FF6DFB612DF]
	(No symbol) [0x00007FF6DFB62441]
	GetHandleVerifier [0x00007FF6E001C76D+3377613]
	GetHandleVerifier [0x00007FF6E0067B67+3685831]
	GetHandleVerifier [0x00007FF6E005CF8B+3641835]
	GetHandleVerifier [0x00007FF6DFDAB2A6+816390]
	(No symbol) [0x00007FF6DFC6B25F]
	(No symbol) [0x00007FF6DFC67084]
	(No symbol) [0x00007FF6DFC67220]
	(No symbol) [0x00007FF6DFC5607F]
	BaseThreadInitThunk [0x00007FFDF6B753E0+16]
	RtlUserThreadStart [0x00007FFDF7B6485B+43]


### Cleaning Data

There are a few issues with datatypes in the above data, so before I proceed I will clean it.

Firstly, all columns are being passed as object datatypes where some should be numeric.
Secondly, there is an issue where an invalid version of the "-" sign is being used.

In [17]:
def clean_data(df):
    int_cols = ['Mins','Shots','SoT','SiB','G','npG','CC','A','P','T','TPA','C 1/3','C PA']
    float_cols = ['Cost','Own','xG','npxG','xA','xGI','npxGI','xP','P vs xP']
    
    for col in df:
        df[col] = df[col].str.replace('−','-')
        if col in int_cols:
            df[col] = df[col].astype(int)
        elif col in float_cols:
            df[col] = df[col].astype(float).round(1)

    return df

df = clean_data(df)

In [18]:
def select_squad(player_data, metric):
    """
    Takes player data and selects a 15 man squad according to the following constraints:
    - 2 gks
    - 5 defs
    - 5 mids
    - 3 atts

    Players are selected in order of a given metric.
    """
    # Define constraints by position
    max_players = {
        'gk':2,
        'def':5,
        'mid':5,
        'att':3,
    }

    # Initialise squad
    squad = {
        'gk':[],
        'def':[],
        'mid':[],
        'att':[]
    }

    # Order player data by given metric
    player_data = player_data.sort_values(by=metric,ascending=False)
    # Initialise count of players in squad
    players_selected = 0

    for index,row in player_data.iterrows():
        # Ensure max players for given position has not been reached
        if len(squad[row['Pos']]) < max_players[row['Pos']]:
            # Create player info dictionary
            player_info = {
                'Player':row['Player'],
                'Team':row['Team'],
                'Cost':row['Cost'],
                'Points':row['P']
            }
            # Add player info to squad
            squad[row['Pos']].append(player_info)
            players_selected += 1
        # Break when squad is filled
        if players_selected == 15:
            return squad

squad = select_squad(df,'P')
squad

{'gk': [{'Player': 'Sánchez', 'Team': 'Chelsea', 'Cost': 4.7, 'Points': 27},
  {'Player': 'Onana', 'Team': 'Manchester United', 'Cost': 5.0, 'Points': 25}],
 'def': [{'Player': 'De Ligt',
   'Team': 'Manchester City',
   'Cost': 15.3,
   'Points': 24},
  {'Player': 'Andersen', 'Team': 'Arsenal', 'Cost': 6.2, 'Points': 23},
  {'Player': 'Konsa', 'Team': 'Chelsea', 'Cost': 7.9, 'Points': 22},
  {'Player': 'Dalot', 'Team': 'Nottingham Forest', 'Cost': 4.5, 'Points': 20},
  {'Player': 'Konaté', 'Team': 'Liverpool', 'Cost': 5.2, 'Points': 17}],
 'mid': [{'Player': 'Palmer', 'Team': 'Chelsea', 'Cost': 10.7, 'Points': 36},
  {'Player': 'McNeil', 'Team': 'Everton', 'Cost': 5.5, 'Points': 29},
  {'Player': 'Martinelli', 'Team': 'Arsenal', 'Cost': 6.9, 'Points': 21},
  {'Player': 'Luis Díaz', 'Team': 'Liverpool', 'Cost': 8.0, 'Points': 21},
  {'Player': 'Barnes', 'Team': 'Newcastle United', 'Cost': 6.5, 'Points': 20}],
 'att': [{'Player': 'Watkins',
   'Team': 'Aston Villa',
   'Cost': 9.1,
   '

In [19]:
def calc_squad_cost(squad):
    """Calculate cost of squad"""
    squad_cost = 0
    for pos in squad:
        for player_info in squad[pos]:
            squad_cost += float(player_info['Cost']).round(1)
    return squad_cost

squad_cost = calc_squad_cost(squad)
squad_cost

AttributeError: 'float' object has no attribute 'round'

## Applying Constraints to Squad

Having written the function to select a squad based on a single metric, I now need to constrain the squad selection according to the rules of FPL. The two key constraints here are:
- No more than 3 players from a given team
- The squad must cost less than the price cap (default £100 million)

### No more than 3 players from the same team

In order to apply this constraint, I will record the teams of each player added. In the same way I skip a player if the number of players in their position is filled, I will also skip players when there are already 3 players from their team.

In [20]:
def select_squad(player_data, metric):
    """
    Takes player data and selects a 15 man squad according to the following constraints:
    - 2 gks
    - 5 defs
    - 5 mids
    - 3 atts
    - Max 3 players from one team

    Players are selected in order of a given metric.
    """
    # Define constraints by position
    max_players = {
        'gk':2,
        'def':5,
        'mid':5,
        'att':3,
    }

    # Initialise squad
    squad = {
        'gk':[],
        'def':[],
        'mid':[],
        'att':[]
    }

    # Initialise count of players from each team
    teams = player_data['Team'].unique()
    players_from_team = dict.fromkeys(teams,0)

    # Order player data by given metric
    player_data = player_data.sort_values(by=metric,ascending=False)
    # Initialise count of players in squad
    players_selected = 0

    for index,row in player_data.iterrows():
        # Ensure max players for given position has not been reached
        if (len(squad[row['Pos']]) < max_players[row['Pos']]) & (players_from_team[row['Team']] < 3):
            # Create player info dictionary
            player_info = {
                'Player':row['Player'],
                'Team':row['Team'],
                'Cost':row['Cost'],
                'Points':row['P']
            }
            # Add player info to squad
            squad[row['Pos']].append(player_info)
            players_selected += 1
            players_from_team[row['Team']] += 1
        # Break when squad is filled
        if players_selected == 15:
            return squad

# squad = select_squad(df,'P')
# print(calc_squad_cost(squad))
# squad

### Constraining Total Team Value

It turns out that picking the subset of players with the maximum number of points whose total cost remains under a given threshold is a variation of a common computer science problem known as the "knapsack problem". More info on this can be found on wikipedia: https://en.wikipedia.org/wiki/Knapsack_problem

There is a Python package that provides a knapsack problem solver: https://developers.google.com/optimization/pack/knapsack#:~:text=In%20the%20knapsack%20problem%2C%20you,can't%20pack%20them%20all.

Maximising the total points, whilst constraining by max players per team, max players per position, and total cost is an advanced version of the knapsack problem. Therefore, I have cheated and asked chatgpt for help.

In [21]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus, PULP_CBC_CMD

def select_fpl_squad(df,metric):
    """
    Selects the optimal 15-man squad based on the given constraints.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing player data with columns 'Player', 'Cost', 'P', 'Team', 'Pos'.
    
    Returns:
    pd.DataFrame: DataFrame containing the selected squad.
    """
    # Reset index to ensure it ranges from 0 to N-1
    df = df.reset_index(drop=True)
    player_ids = df.index.tolist()

    # Convert 'Cost' and 'P' columns to numeric
    df['Cost'] = pd.to_numeric(df['Cost'])
    df[metric] = pd.to_numeric(df[metric])

    # Define the LP problem
    prob = LpProblem("FPL_Squad_Selection", LpMaximize)

    # Define binary decision variables for each player
    x = LpVariable.dicts('x', player_ids, cat='Binary')

    # Objective function: maximize total points
    prob += lpSum(df.loc[i, metric] * x[i] for i in player_ids), "Total_Points"

    # Total cost constraint
    prob += lpSum(df.loc[i, 'Cost'] * x[i] for i in player_ids) <= 100, "Total_Cost"

    # Position constraints
    positions = {'gk': 2, 'def': 5, 'mid': 5, 'att': 3}
    for pos, count in positions.items():
        pos_ids = df[df['Pos'] == pos].index.tolist()
        prob += lpSum(x[i] for i in pos_ids) == count, f"Total_{pos.upper()}"

    # Team constraints: no more than 3 players from each team
    for team in df['Team'].unique():
        team_ids = df[df['Team'] == team].index.tolist()
        prob += lpSum(x[i] for i in team_ids) <= 3, f"Team_{team}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=0))

    # Check if an optimal solution was found
    if LpStatus[prob.status] != 'Optimal':
        print("No optimal solution found.")
        return None

    # Get the selected players
    selected_ids = [i for i in player_ids if x[i].varValue == 1]
    selected_squad = df.loc[selected_ids].reset_index(drop=True)

    return selected_squad

# Assuming 'data' is your DataFrame containing the player data
metric = 'xP'
optimal_squad = select_fpl_squad(df,metric)

In [22]:
optimal_squad

,Player,Team,Cost,Own,Mins,Shots,SoT,SiB,xG,npxG,...,xGI,npxGI,xP,P,P vs xP,T,TPA,C 1/3,C PA,Pos
0,Raya,Arsenal,5.6,30.9,270,0,0,0,0.0,0.0,...,0.0,0.0,16.7,14,-2.7,108,0,0,0,gk
1,Sánchez,Chelsea,4.7,11.4,270,0,0,0,0.0,0.0,...,0.0,0.0,16.3,27,10.7,167,0,0,0,gk
2,Gabriel,Tottenham,7.6,9.6,267,10,5,9,3.1,3.1,...,3.3,3.3,24.0,14,-10.0,88,18,2,4,def
3,Andersen,Arsenal,6.2,24.8,270,6,4,6,1.1,1.1,...,1.1,1.1,21.3,23,1.7,167,7,1,0,def
4,Konsa,Chelsea,7.9,17.0,219,12,4,11,2.1,2.1,...,2.8,2.8,20.5,22,1.5,76,15,2,7,def
5,Bassey,Fulham,4.5,0.4,270,2,0,1,0.1,0.1,...,0.1,0.1,13.8,10,-3.8,170,1,0,0,def
6,Tete,Fulham,4.5,0.4,270,6,0,6,0.4,0.4,...,0.4,0.4,12.6,8,-4.6,190,13,4,7,def
7,M.Salah,Liverpool,12.8,42.0,270,10,7,9,2.1,1.3,...,2.9,2.1,26.0,18,-8.0,130,30,2,15,mid
8,Palmer,Chelsea,10.7,44.5,243,9,6,7,2.5,1.7,...,3.5,2.7,25.3,36,10.7,108,8,5,2,mid
9,Kulusevski,Tottenham,6.2,1.4,270,9,3,7,1.0,1.0,...,2.6,2.6,18.9,14,-4.9,144,21,7,6,mid


In [23]:
def calc_squad_cost(squad_df):
    return float(squad_df['Cost'].round(1).sum())

calc_squad_cost(optimal_squad)

99.89999999999999

In [ ]:
def transfers(old_squad,new_squad):
    players_out = [player for player in old_squad['Player'] if player not in new_squad['Player']]
    players_in = [player for player in new_squad['Player'] if player not in old_squad['Player']]
    return players_out, players_in